In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os.path
import os
import json
from datetime import datetime
from datetime import timedelta


# https://api.slack.com/apps/APSE1SZPZ/incoming-webhooks?
class SlackNotifier():
    def __init__(self, url, interval_in_s):
        self.slack_hook_url = url
        self.tdelta = timedelta(seconds=interval_in_s)
        self.lasttime = {}
    def call(self, Id, message, forced=False):
        if forced:
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
        elif not self.is_just_called(Id):
            requests.post(self.slack_hook_url, json={"text":message})
            self.lasttime[Id] = datetime.now()
    def is_just_called(self, Id):
        if not self.lasttime:
            return False
        elif Id in self.lasttime.keys():
            return datetime.now() < self.lasttime[Id] + self.tdelta        
        else:
            return False
    
def notify_slack(message):
    global updated
    print(updated)
    if not updated:
        last_notification_time = datetime.now()
        updated = True
        print("updated")
    print(last_notification_time)
    slack_hook_url = os.environ['SLACK_HOOK_URL']
    requests.post(slack_hook_url, json={"text":message})


def extract_today_created_time(item_time):
    if "Heute" in item_time:
        index = item_time.find("Heute,")
        created_hours = int(item_time[index+7:index + 9])
        created_minutes = int(item_time[index+10:index + 12])
        return True, timedelta(hours=created_hours, minutes=created_minutes)
    else:
        return False, item_time.replace("\n","").replace(" ","")
    
def is_new(created_time, interval_in_s=30000):
    """ Post can be updated later in 3 minutes => use default interval is 4' = 240"
    """
    tdelta = timedelta(seconds=int(interval_in_s))
    now_hours = datetime.now().hour
    now_minutes = datetime.now().minute
    now_seconds = datetime.now().second
    now = timedelta(hours=now_hours, minutes=now_minutes, seconds=now_seconds)
    return now < created_time + tdelta

In [2]:
notifier = SlackNotifier(os.environ['SLACK_HOOK_URL'], 10)

In [3]:
notifier.call(0, "FOR EBAY-KLEINANZEIGEN: NEW MESSAGE.")

In [63]:
lat = "48.1151649"
lng = "11.6981558"
queries = ["ikea skarsta", "bose revolve"]
queries = ["+".join(query.split(" ")) for query in queries]
distances = [20, 20] # in km
maxprices = [110, 110] # in km
page_num = 0
interval_in_s = 30

assert len(distances) == len(queries), "Number of queries and distance is different."
assert len(queries) == len(maxprices), "Number of queries and maxprice is different."

In [64]:
i = 1
query = queries[i]
dist = distances[i]
maxprice = maxprices[i]

In [66]:
url_for_location_id = "https://m.ebay-kleinanzeigen.de/s-ort-vorschlag.json?lat="+ lat + "&lng=" + lng
result = requests.get(url_for_location_id)
page = result.text
locationId = json.loads(page)[0]['id']

In [67]:
url = "https://m.ebay-kleinanzeigen.de/s-suche-veraendern?locationId=" + str(locationId) + \
        "&distance=" + str(dist) + \
        "&categoryId=&minPrice=&maxPrice=" + str(maxprice) + \
        "&adType=&posterType="+ \
        "&q=" + query

result = requests.get(url)
page = result.text
doc = soup(page, "html.parser")
links = [element.get('content') for element in doc.find_all('meta')]
link = links[6]

link0, link1 = link.split("distance")
url = link0 + "page=" + str(page_num) + "&distance" + link1

In [69]:
result = requests.get(url)
page = result.text
doc = soup(page, "html.parser")
items = [element for element in doc.find_all('div', {"class": "adlist--item--descarea"})]

In [70]:
items

[]

In [71]:
if len(items) > 0:
    item = items[0] # get last item
    item_time = item.find('div', {"class": "adlist--item--info--date"}).contents[0]
    is_today, created_time = extract_today_created_time(item_time)
    if is_today:
        print("Today {}".format(created_time))
        if is_new(created_time):
            notifier.call("FOR EBAY-KLEINANZEIGEN: NEW MESSAGE.", forced=False)
            print("SEND")
    else:
        print(created_time)
            

# Delete process script

In [54]:
import os

if os.path.exists("/tmp/ebay-kleinanzeigen.pid"):
    with open("/tmp/ebay-kleinanzeigen.pid", "r") as f:
        current_pid = f.readline()
        current_pid = current_pid.split("\n")[0]
        kill_syntax = "kill -9 {}".format(current_pid)
        os.system(kill_syntax)
        if os.path.exists("/tmp/ebay-kleinanzeigen.pid.lock"):
            os.remove("/tmp/ebay-kleinanzeigen.pid.lock")